In [0]:
%pip install unitycatalog-ai[databricks]
dbutils.library.restartPython()

In [0]:
from unitycatalog.ai.core.databricks import DatabricksFunctionClient

client = DatabricksFunctionClient()

In [0]:

CATALOG = "eo000_ctg"
SCHEMA = "fs_db"

def python_exec(code: str) -> str:
    """
    Executes Python code in a stateless sandboxed environment and returns its stdout. The runtime cannot access files or read previous executions' output. All operations must be self-contained, using only standard Python libraries. Calls to other tools are prohibited.

    Args:
        code (str): Python code to execute. Ensure that all variables are initialized within the code, and import any necessary standard libraries. The code must print the final result to stdout. Do not attempt to access files or external systems.

    Returns:
        str: The stdout of the executed code.
    """
    import sys
    from io import StringIO
    stdout = StringIO()
    stderr = StringIO()
    sys.stdout = stdout
    sys.stderr = stderr
    try:
        exec(code, {})
    except SyntaxError as e:
        def get_string_version(code_str):
            return code_str.encode('utf-8').decode('unicode_escape')
        code = get_string_version(code)
        exec(code, {})
    return stdout.getvalue() + stderr.getvalue()

function_info = client.create_python_function(
  func=python_exec,
  catalog=CATALOG,
  schema=SCHEMA,
  replace=True
)